In [1]:
## data
import pandas as pd
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta
from unicodedata import normalize

## vis
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

## modelo
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

from treeinterpreter import treeinterpreter as ti

## alertas
import warnings
warnings.filterwarnings("ignore")

In [2]:
PATH = "datasets/"

In [3]:
FILE = "ALIADOS_KPI.csv"
nome_fornecedores = pd.read_csv(PATH + FILE, sep = ';', encoding = 'ISO-8859-1', engine ='python')
nome_fornecedores = nome_fornecedores[['RAIZ_CNPJ', 'NOME_FORNECEDOR']]

nome_fornecedores['RAIZ_CNPJ'] = nome_fornecedores['RAIZ_CNPJ'].apply(lambda x: str(x).zfill(8))
nome_fornecedores = nome_fornecedores[nome_fornecedores["RAIZ_CNPJ"].str.contains("NAN")== False]
nome_fornecedores = nome_fornecedores[nome_fornecedores["RAIZ_CNPJ"].str.contains("-")== False]

for i in nome_fornecedores.columns:
    if nome_fornecedores[i].dtypes=='object':
        nome_fornecedores[i] = nome_fornecedores[i].str.upper()
        
nome_fornecedores = nome_fornecedores.drop_duplicates().reset_index(drop=True)

nome_fornecedores

,RAIZ_CNPJ,NOME_FORNECEDOR
0,00000000,HODNET S/A
1,00000B60,ITP TECNOLOGICA SL
2,00000B86,"DELOITTE ADVISORY, S.L."
3,00028986,ELEVADORES ATLAS SCHINDLER SA
4,00059799,HDS SISTEMAS DE ENERGIA LTDA
...,...,...
1427,44522486,BUENO CONSULTORIA TELECON LTDA
1428,46325566,SOARES COMUNICACOES LTDA
1429,59844621,MICROFONE COMUNICAÃÃES ARTISTICAS L
1430,73092132,RBC INFORMAÃÃES ELETRÃNICAS


In [4]:
len(nome_fornecedores['NOME_FORNECEDOR'].unique())

1427

In [5]:
FILE = '20220923 - CAD_FORNECEDORES.csv'
fornecedores = pd.read_csv(PATH + FILE, sep = ';', dtype=str, encoding = 'ISO-8859-1')

# renomea RAIZ_CNPJ
fornecedores = fornecedores.rename(columns={'RAIZ CNPJ': 'RAIZ_CNPJ'})

# seleciona colunas
fornecedores = fornecedores[['RAIZ_CNPJ', 'FORNECEDOR']]

fornecedores['FORNECEDOR'] = fornecedores['FORNECEDOR'].apply(lambda x: str(x).zfill(8))
fornecedores['RAIZ_CNPJ'] = fornecedores['RAIZ_CNPJ'].apply(lambda x: str(x).zfill(8))
fornecedores = fornecedores[fornecedores["RAIZ_CNPJ"].str.contains("NAN")== False]
fornecedores = fornecedores[fornecedores["RAIZ_CNPJ"].str.contains("-")== False]

fornecedores = fornecedores.drop_duplicates().reset_index(drop=True)\
                           .drop(fornecedores.index[0])

fornecedores['FORNECEDOR'] = fornecedores['FORNECEDOR'].fillna(value = '0')\
                            .apply(lambda x: x.replace('.', '').replace(',', '').replace('-', '').replace(' ', ''))

fornecedores

,RAIZ_CNPJ,FORNECEDOR
1,67431718,14003148
2,10414014,8007109
3,34639119,14066918
4,08783936,10016619
5,66869520,8010675
...,...,...
11491,33279435,14066745
11492,04738496,7714093
11493,01451373,14024176
11494,17197385,8005781


In [7]:
len(fornecedores['RAIZ_CNPJ'].unique())

6767

In [8]:
PATH_CNAE = '../data/20221207_etl/'
FILE = "FornecedoresArea.csv"
cnaes = pd.read_csv(PATH_CNAE + FILE, sep = ';', encoding = 'ISO-8859-1', engine ='python')
cnaes = cnaes.rename(columns={'ï»¿RAIZ_CNPJ': 'RAIZ_CNPJ'})
cnaes = cnaes[['RAIZ_CNPJ', 'AREA_ATUACAO']]

cnaes['RAIZ_CNPJ'] = cnaes['RAIZ_CNPJ'].apply(lambda x: str(x).zfill(8))
cnaes = cnaes[cnaes["RAIZ_CNPJ"].str.contains("NAN")== False]
cnaes = cnaes[cnaes["RAIZ_CNPJ"].str.contains("-")== False]

for i in cnaes.columns:
    if cnaes[i].dtypes=='object':
        cnaes[i] = cnaes[i].str.upper()
        
cnaes = cnaes.drop_duplicates().reset_index(drop=True)

cnaes

,RAIZ_CNPJ,AREA_ATUACAO
0,67431718,CULTURA ESPORTES E EDUCAÃÃO
1,10414014,CULTURA ESPORTES E EDUCAÃÃO
2,34639119,CULTURA ESPORTES E EDUCAÃÃO
3,08783936,TELECOMUNICAÃÃES
4,66869520,COMUNICACAO E MARKETING
...,...,...
6779,18748007,INDUSTRIA E AGRO
6780,33279435,INFORMATICA
6781,04738496,COMÃRCIO
6782,01451373,INFORMATICA


In [9]:
len(cnaes['RAIZ_CNPJ'].unique())

6757

In [10]:
fornecedores_cnae = nome_fornecedores.join(cnaes.set_index('RAIZ_CNPJ'),
                                           how='inner',
                                           on=['RAIZ_CNPJ'])\
                                     .drop_duplicates().reset_index(drop=True)

fornecedores_cnae

,RAIZ_CNPJ,NOME_FORNECEDOR,AREA_ATUACAO
0,00000000,HODNET S/A,ATIVIDADES FINANCEIRAS
1,00028986,ELEVADORES ATLAS SCHINDLER SA,CONSTRUÃÃO
2,00059799,HDS SISTEMAS DE ENERGIA LTDA,INDUSTRIA E AGRO
3,00078599,GERENCIAL BRASIL PARTICIPAÃÃES E EM,RECURSOS HUMANOS
4,00167629,"CAVALLAZZI, ANDREY, RESTANHO & ARAU",ADVOCACIA
...,...,...,...
1318,37415516,TAS - TECNOLOGIA APLICADA A SEGURAN,INFORMATICA
1319,44378838,NK 147 EMPREENDIMENTOS E PARTICIPAÃ,CULTURA ESPORTES E EDUCAÃÃO
1320,59844621,MICROFONE COMUNICAÃÃES ARTISTICAS L,SERVIÃOS
1321,73092132,RBC INFORMAÃÃES ELETRÃNICAS,SERVIÃOS


In [11]:
len(fornecedores_cnae['RAIZ_CNPJ'].unique())

1319

In [47]:
fornecedor_final = fornecedores.join(fornecedores_cnae.set_index('RAIZ_CNPJ'),
                                     how='inner',
                                     on=['RAIZ_CNPJ'])\
                               .drop_duplicates().reset_index(drop=True)

fornecedor_final

,RAIZ_CNPJ,FORNECEDOR,NOME_FORNECEDOR,AREA_ATUACAO
0,02714643,7700312,333 COMERCIO E COMUNICACOES LTDA,COMÃRCIO
1,02714643,7700251,333 COMERCIO E COMUNICACOES LTDA,COMÃRCIO
2,02714643,7716773,333 COMERCIO E COMUNICACOES LTDA,COMÃRCIO
3,02714643,7721785,333 COMERCIO E COMUNICACOES LTDA,COMÃRCIO
4,02714643,7729957,333 COMERCIO E COMUNICACOES LTDA,COMÃRCIO
...,...,...,...,...
4572,08747596,7729259,ZOTTY CELULARES LTDA - ME,COMÃRCIO
4573,08747596,7729332,ZOTTY CELULARES LTDA - ME,COMÃRCIO
4574,05216804,14017703,ZTE DO BRASIL INDÃSTRIA COMÃRCIO,TELECOMUNICAÃÃES
4575,05216804,14025498,ZTE DO BRASIL INDÃSTRIA COMÃRCIO,TELECOMUNICAÃÃES


In [48]:
fornecedor_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4577 entries, 0 to 4576
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   RAIZ_CNPJ        4577 non-null   object
 1   FORNECEDOR       4577 non-null   object
 2   NOME_FORNECEDOR  4577 non-null   object
 3   AREA_ATUACAO     4577 non-null   object
dtypes: object(4)
memory usage: 143.2+ KB


In [49]:
len(fornecedor_final['RAIZ_CNPJ'].unique())

1319

In [50]:
fornecedor_final.to_excel(r'datasets/nome_fornecedores.xlsx')

In [51]:
# PATH_TESTE = '../data/20221207_etl/'

In [52]:
# FILE = "FornecedoresArea.csv"
# cnaes = pd.read_csv(PATH_TESTE + FILE, sep = ';', encoding = 'ISO-8859-1', engine ='python')
# cnaes = cnaes.rename(columns={'ï»¿RAIZ_CNPJ': 'RAIZ_CNPJ'})
# cnaes = cnaes[['RAIZ_CNPJ', 'AREA_ATUACAO']]

# cnaes['RAIZ_CNPJ'] = cnaes['RAIZ_CNPJ'].apply(lambda x: str(x).zfill(8))
# cnaes = cnaes[cnaes["RAIZ_CNPJ"].str.contains("NAN")== False]
# cnaes = cnaes[cnaes["RAIZ_CNPJ"].str.contains("-")== False]

# for i in cnaes.columns:
#     if cnaes[i].dtypes=='object':
#         cnaes[i] = cnaes[i].str.upper()
        
# cnaes = cnaes.drop_duplicates().reset_index(drop=True)

# cnaes

In [53]:
# len(teste_csv['AREA_ATUACAO'].unique())